In [1]:
import pandas as pd
import plotly.express as px
import plotly
import glob
import plotly.io as pio
pio.renderers.default = "notebook_connected"

plotly.offline.init_notebook_mode(connected=True)
plotly.io.templates.default = "plotly_white"

# Data Loading and Aggregating

In [2]:
DATA_PATH = "/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/"

In [3]:
def load_and_convert(path):
    """load one dataset from the path and convert date to timestamp"""
    df = pd.read_csv(path)
    return df.assign(publish_date=pd.to_datetime(df["publish_date"]))

In [4]:
all_csv_paths = glob.glob(DATA_PATH + "*dedup*")
all_csv_paths = [csv_path for csv_path in all_csv_paths if ("nytimes" not in csv_path)]
all_csv_paths

['/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/newyork_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/florida_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/illinois_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/texas_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/california_article_texts_and_info_dedup.csv',
 '/home/pranavgoel/trans-fer-entropy/obtaining_news_collections/data/ohio_article_texts_and_info_dedup.csv']

In [5]:
def num_articles_per_media(df, state, temporal=None):
    """Take a state df and then group by its media name and count the number of articles per media.
    You can also supply the temporal grouping here. Please use the use this parameter to specify the frequency of temporal grouping.
    See eligible values: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    """

    if temporal is not None:
        num_news_per_media = (
            df.groupby([pd.Grouper(key="publish_date", freq=temporal), "media_name"])
            .size()
            .reset_index(name="count")
        )
        num_news_per_media["state"] = state
    else:
        num_news_per_media = df.groupby("media_name").size().reset_index(name="count")
        num_news_per_media["state"] = state

    return num_news_per_media

In [6]:
num_articles_per_media_df = []
num_articles_per_media_temporal_df = []
for file_path in all_csv_paths:
    state = file_path.split("/")[-1].split("_")[0]
    state_df = load_and_convert(file_path)
    

    num_total_media = state_df["media_name"].nunique()
    print(f"Loading files from {state}: {num_total_media} unique meida found")

    # getting # articles per media
    num_articles_per_media_df.append(num_articles_per_media(state_df, state))
    num_articles_master = pd.concat(num_articles_per_media_df)

    # getting daily # articles per media
    num_articles_per_media_temporal_df.append(
        num_articles_per_media(state_df, state, temporal="D")
    )
    num_articles_temporal_master = pd.concat(num_articles_per_media_temporal_df)

Loading files from newyork: 101 unique meida found
Loading files from florida: 54 unique meida found
Loading files from illinois: 48 unique meida found
Loading files from texas: 85 unique meida found
Loading files from california: 253 unique meida found
Loading files from ohio: 41 unique meida found


In [7]:
collection_color_map = {
    "nyt": "#2E498A",
    "newyork": "#4B2D6B",
    "illinois": "#72C29F",
    "california": "#64CDF0",
    "fox": "#7E1939",
    "texas": "#F5CF65",
    "ohio": "#EF9173",
    "florida": "#D68537",
}

In [11]:
num_articles_temporal_master['state'].unique()

array(['newyork', 'florida', 'illinois', 'texas', 'california', 'ohio'],
      dtype=object)

# Overall Distribution 

## Number of Articles per Media 

In [32]:
fig_voilin=px.violin(
    num_articles_master,
    box=True,
    points="all",
    hover_data=num_articles_master.columns,
    color="state",
    color_discrete_map=collection_color_map,
    labels={'state':'State','count':'Count'},
    # height=900,
    x="count",
    y="state",
).update_layout(showlegend=False)

fig_voilin.write_image('fig_violin.png', scale=2)
fig_voilin.write_image('fig_violin.svg')

fig_voilin


## Normalized Count

In [16]:
num_articles_master["perc"] = (num_articles_master["count"]) / (
    num_articles_master.groupby(["state"])["count"].transform("sum")
)

In [33]:
fig_voilin_norm=px.violin(
    num_articles_master,
    box=True,
    points="all",
    hover_data=num_articles_master.columns,
        color_discrete_map=collection_color_map,
        labels={'state':'State','count':'Count','perc':'Percent'},
    color="state",
    # height=900,
    x="perc",
    y="state",
).update_layout(showlegend=False)

fig_voilin_norm.write_image('fig_violin_pct.png', scale=2)
fig_voilin_norm.write_image('fig_violin_pct.svg')

fig_voilin_norm

In [43]:
fig_his=px.histogram(
    num_articles_master,
    #   box=True,
    #   points='all',
    hover_data=num_articles_master.columns,
        color_discrete_map=collection_color_map,
    histnorm="probability density",
    color="state",
     labels={'state':'State','count':'Count','perc':'Percent','probability density':'Prob.'},
    height=700,
    x="count",
    facet_row="state",
)

fig_his.update_layout(showlegend=False).update_yaxes(title='Prob.')


fig_his.write_image('fig_violin_pct.png', scale=2)
fig_his.write_image('fig_violin_pct.svg')

fig_his

In [20]:
px.histogram(
    num_articles_master,
    #   box=True,
    #   points='all',
    hover_data=num_articles_master.columns,
    marginal="rug",
    histnorm="percent",
    color="state",
    barmode="overlay",
    height=700,
    x="count",
)

## Who are the those media? (Querying media with >0.1 prevelance)

In [44]:
px.colors.sequential

<module '_plotly_utils.colors.sequential' from '/home/crazybrokeasian/yukun_env/lib/python3.8/site-packages/_plotly_utils/colors/sequential.py'>

In [56]:
dominant_media=px.bar(
    num_articles_master.query("perc >= 0.1"),
    y="media_name",
    facet_row="state",
    height=700,
    color="perc",    color_discrete_map=collection_color_map,
    color_continuous_scale=px.colors.sequential.Viridis,
    hover_data=num_articles_master.columns,
    labels={'perc':'Percent'},
    x="perc",
).update_yaxes(matches=None, title='Outlet').update_xaxes(tickformat='.0%').update_layout(coloraxis_colorbar={
    'tickformat':'.0%'
})


dominant_media.write_image('fig_bar_dominant.png', scale=2)
dominant_media.write_image('fig_bar_dominant.svg')

dominant_media

# Temporal Changes

## Number of News Daily

In [63]:
fig_temporal_line=px.line(num_articles_temporal_master,
        x='publish_date',
        y='count',
        color='media_name',
        height=800,
            color_discrete_map=collection_color_map,
            labels={'publish_date':'Publication Date'},
        facet_row='state').update_layout(showlegend=False).update_yaxes(matches=None)

fig_temporal_line.update_yaxes(title='Count')



fig_temporal_line.write_image('fig_temporal_line.png', scale=2)
fig_temporal_line.write_image('fig_temporal_line.svg')

fig_temporal_line

## Normalized Daily Count

In [59]:
num_articles_temporal_master_norm=num_articles_temporal_master.groupby(["state", "publish_date"]).apply(
    lambda x: x.assign(perc=x['count']/x['count'].sum())
).reset_index(drop=True)

In [60]:
num_articles_temporal_master_norm.head()

,publish_date,media_name,count,state,perc
0,2023-01-01,chicoer.com,1,california,0.111111
1,2023-01-01,dailykos.com,1,california,0.111111
2,2023-01-01,easyreadernews.com,1,california,0.111111
3,2023-01-01,latimes.com,1,california,0.111111
4,2023-01-01,mu.nu,1,california,0.111111


In [64]:
fig_area_daily=px.area(num_articles_temporal_master_norm,
       x='publish_date',
       y='perc',
       color='media_name',
       height=900,
       labels={'perc':'Percent', 'publish_date':'Publication Date'},
       facet_row='state').update_layout(showlegend=False)



fig_area_daily.write_image('fig_area_daily.png', scale=2)
fig_area_daily.write_image('fig_area_daily.svg')

fig_area_daily